 # **Torgo Speakers spelling correction with machine translation techniques**


### **Objective: Spelling correction for F01 Torgo speakers using machine translation techniques**

### **Ensure that GPU and RAM is set up: will be needed for training purpose**

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Jun 11 05:06:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# ensure enough memory present so that training does not stop
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


### **Install the libraries**

In [ ]:
# Install required libraries
!pip install datasets
!pip install transformers==4.28.0
!pip install accelerate
!pip install jiwer
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 23.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 72.8 MB/s eta 0:00

### **Import libraries**

In [ ]:
# Import libraries
import torch
from transformers import BartTokenizerFast, BartForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from jiwer import wer
from huggingface_hub import notebook_login

In [ ]:
# Login to Hugging Face
notebook_login()

### **Mount the json files from Google Drive**

In [ ]:
# mount other_speakers.json file is stored in Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Load the data and set up the tokenizer**

In [ ]:
# Load the dataset from the JSON files
dataset = load_dataset('json', data_files={'train': '/content/drive/MyDrive/f01_other_speakers.json',
                                           'test': '/content/drive/MyDrive/speaker_f01.json'})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-9c0e3ac61925a266/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['path', 'speaker', 'prediction', 'actual'],
        num_rows: 5212
    })
    test: Dataset({
        features: ['path', 'speaker', 'prediction', 'actual'],
        num_rows: 215
    })
})


In [ ]:
print(dataset['train'][:5])
print(dataset['test'][:5])

{'path': ['content/downloads/Torgo/F03/Session1/wav_arrayMic/0005.wav', 'content/downloads/Torgo/F03/Session1/wav_arrayMic/0006.wav', 'content/downloads/Torgo/F03/Session1/wav_arrayMic/0007.wav', 'content/downloads/Torgo/F03/Session1/wav_arrayMic/0008.wav', 'content/downloads/Torgo/F03/Session1/wav_arrayMic/0009.wav'], 'speaker': ['F03', 'F03', 'F03', 'F03', 'F03'], 'prediction': ['beta', 'stubble', 'stubble', 'trace', 'goat'], 'actual': ['beta ', 'stubble ', 'stubble ', 'trace ', 'goat ']}
{'path': ['content/downloads/Torgo/F01/Session1/wav_arrayMic/0006.wav', 'content/downloads/Torgo/F01/Session1/wav_arrayMic/0008.wav', 'content/downloads/Torgo/F01/Session1/wav_arrayMic/0009.wav', 'content/downloads/Torgo/F01/Session1/wav_arrayMic/0010.wav', 'content/downloads/Torgo/F01/Session1/wav_arrayMic/0012.wav'], 'speaker': ['F01', 'F01', 'F01', 'F01', 'F01'], 'prediction': ['snick', 'ealsein the winu we wol  orice pl', 'pat', 'up', 'nit'], 'actual': ['stick ', 'except in the winter when the o

By setting a fixed random seed, the data will be split into training and validation sets in a consistent manner each time the code is executed. This is useful for debugging, testing, and comparing different runs of the code. The choice of the number 42 as the seed is arbitrary and can be any integer value. The important aspect is to use the same seed consistently if reproducibility is desired.

In [ ]:
# Split the dataset into train and validation sets
train_dataset, val_dataset = train_test_split(dataset['train'], test_size=0.1, random_state=42)

In [ ]:
from datasets import Dataset

train_data = Dataset.from_dict(train_dataset)  # Convert the train data to a dataset
val_data = Dataset.from_dict(val_dataset)      # Convert the validation data to a dataset

In [ ]:
print(type(train_data))
print(type(val_data))

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


In [ ]:
# Check column names in train dataset
print(train_data.column_names)

# Check column names in validation dataset
print(val_data.column_names)

['path', 'speaker', 'prediction', 'actual']
['path', 'speaker', 'prediction', 'actual']


In [ ]:
print(train_data['actual'][:5])
print(train_data['prediction'][:5])
#print(val_data[:5])

['no ', 'bloat ', 'liar ', 'race ', 'fee ']
['no', 'bloat', 'liar', 'race', 'fee']


In [ ]:
# Define the source and target language columns
source_lang = 'prediction'
target_lang = 'actual'

In [ ]:
print(source_lang)

prediction


In [ ]:
# Define the max_length for padding and truncation
max_length = 512

The preprocessing function serves to prepare the data for training or evaluation. It uses a tokenizer to tokenize the inputs and labels, formats the inputs by adding a source language identifier, encodes the tokenized inputs and labels, and creates a dictionary of model inputs. The function ensures that the data is properly tokenized, formatted, and encoded according to the model's requirements. It helps maintain consistency and compatibility between the input data and the model during training or evaluation.

In [ ]:
# Initialize the tokenizer
tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-base')

# Tokenize the data
# The preprocess_function function is defined to preprocess the data by tokenizing the inputs and labels
def preprocess_function(examples):
    inputs = [f'{source_lang}: {text}' for text in examples[source_lang]]
    targets = examples[target_lang]
    encoding = tokenizer(inputs, padding=True, truncation=True, return_tensors='pt', max_length=max_length)
    model_inputs = {
        'input_ids': encoding['input_ids'].squeeze(),
        'attention_mask': encoding['attention_mask'].squeeze(),
        'labels': tokenizer(targets, padding=True, truncation=True, return_tensors='pt')['input_ids'].squeeze()
    }
    return model_inputs


In [ ]:
# Select a random data point from the train dataset
sample_data = train_data[0]

# Call the preprocess function on the sample data
processed_data = preprocess_function(sample_data)

# Inspect the output
print(processed_data)

{'input_ids': tensor([[    0, 37466, 26579,    35,   295,     2],
        [    0, 37466, 26579,    35,  1021,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1]]), 'labels': tensor([   0, 2362, 1437,    2])}


In [ ]:
#train_data = preprocess_function(train_data)
#val_data = preprocess_function(val_data)

# Apply preprocess_function to train_data and val_data
train_data = train_data.map(preprocess_function, batched=True)
val_data = val_data.map(preprocess_function, batched=True)


Map:   0%|          | 0/4690 [00:00<?, ? examples/s]

Map:   0%|          | 0/522 [00:00<?, ? examples/s]

In [ ]:
# Access a few samples from train_data
for i in range(5):
    sample_input_ids = train_data['input_ids'][i]
    sample_attention_mask = train_data['attention_mask'][i]
    sample_labels = train_data['labels'][i]

    print(f"Sample {i+1}:")
    print("Input IDs:", sample_input_ids)
    print("Attention Mask:", sample_attention_mask)
    print("Labels:", sample_labels)
    print()

Sample 1:
Input IDs: [0, 37466, 26579, 35, 117, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Attention Mask: [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Labels: [0, 2362, 1437, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Sample 2:
Input IDs: [0, 37466, 26579, 35, 3089, 34827, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

A data loader is a component used in machine learning frameworks, such as PyTorch, to handle the loading and batching of data during the training or evaluation process. Its main purpose is to efficiently provide batches of data to the model for processing.

In [ ]:
# Prepare DataLoader for training and validation
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=16)

### **Train the model**

The purpose of a data collator is to take a list of samples from a dataset and collate them into a batch that can be processed by the model during training or evaluation. It works closely with the data loader to handle the specific requirements of the model's input format.

In [ ]:
# define a data_collator function for batch processing
def data_collator(features):
    batch = {}
    # Pad input_ids and attention_mask to the maximum length within the batch
    max_length = max(len(feature['input_ids']) for feature in features)
    batch['input_ids'] = torch.stack([torch.tensor(feature['input_ids'] + [tokenizer.pad_token_id] * (max_length - len(feature['input_ids']))) for feature in features])
    batch['attention_mask'] = torch.stack([torch.tensor(feature['attention_mask'] + [0] * (max_length - len(feature['attention_mask']))) for feature in features])
    batch['labels'] = torch.stack([torch.tensor(feature['labels'] + [-100] * (max_length - len(feature['labels']))) for feature in features])
    return batch

The data loader is responsible for loading and batching the data, while the data collator is responsible for formatting and aligning the data within each batch. They serve different functions in the training process.

In [ ]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="spell_correction_F01",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=30,
    predict_with_generate=True,
    push_to_hub=True,
)

In [ ]:
# verify passing the correct inputs to the trainer
print("Train Dataset:", train_data)
print("Validation Dataset:", val_data)
print("Tokenizer:", tokenizer)
print("Training Arguments:", training_args)

Train Dataset: Dataset({
    features: ['path', 'speaker', 'prediction', 'actual', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 4690
})
Validation Dataset: Dataset({
    features: ['path', 'speaker', 'prediction', 'actual', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 522
})
Tokenizer: BartTokenizerFast(name_or_path='facebook/bart-base', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)}, clean_up_tokenization_spaces=True)
Training Arguments: Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,

In [ ]:
# model is initialized with the BARTForConditionalGeneration class and moved to the GPU if available.
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [ ]:
# DO NOT RUN THE CODE BLOCK
# code to keep google collab running:
# Right click -> Inspect -> Console -> paste the code below
function ClickConnect(){
  console.log("Clicking on the page");
  document.querySelector("colab-connect-button").click()
}
setInterval(ClickConnect, 60000) // Clicks every 1 minute

In [ ]:
# The Seq2SeqTrainer is created with the defined model, training arguments, datasets, tokenizer, and data_collator
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

/content/spell_correction_F01 is already a clone of https://huggingface.co/monideep2255/spell_correction_F01. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.191430
2,1.868600,0.037681
3,1.868600,0.023813
4,0.035200,0.022701
5,0.035200,0.018183
6,0.017200,0.016637
7,0.008500,0.015776
8,0.008500,0.015311
9,0.006500,0.013899
10,0.006500,0.013499


Epoch,Training Loss,Validation Loss
1,No log,0.191430
2,1.868600,0.037681
3,1.868600,0.023813
4,0.035200,0.022701
5,0.035200,0.018183
6,0.017200,0.016637
7,0.008500,0.015776
8,0.008500,0.015311
9,0.006500,0.013899
10,0.006500,0.013499


TrainOutput(global_step=8820, training_loss=0.11130070876913006, metrics={'train_runtime': 5314.6331, 'train_samples_per_second': 26.474, 'train_steps_per_second': 1.66, 'total_flos': 9523212936007680.0, 'train_loss': 0.11130070876913006, 'epoch': 30.0})

### **Evaluate and Test the model**

In [ ]:
# Load the trained model from Hugging Face
model = BartForConditionalGeneration.from_pretrained("monideep2255/spell_correction_F01")

In [ ]:
# Move the model to the GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
#model.eval()

In [ ]:
# Test the model on speaker dataset
test_dataset = dataset['test'].filter(lambda example: example['speaker'] == 'F01')
print(len(test_dataset))

In [ ]:
print(test_dataset)

In [ ]:
# Inspect column names
column_names = test_dataset.column_names
print("Column names:", column_names)

# Inspect column data types
for column in column_names:
    column_data = test_dataset[column]
    data_type = type(column_data[0]) if len(column_data) > 0 else "Unknown"
    print(f"Column name: {column}, Data type: {data_type}")

In [ ]:
# verification
# Test the model on speaker dataset
verify_test_dataset = dataset['test']
print(len(verify_test_dataset))
print((verify_test_dataset))

In [ ]:
# Print the samples
for sample in test_dataset:
    print("actual:", sample["actual"])
    print("Speaker ID:", sample["speaker"])
    print("prediction:", sample["prediction"])
    print()

In [ ]:
test_data = test_dataset.map(preprocess_function, batched=True, batch_size=len(test_dataset))
model.eval()

In the following code, we iterate over each example in the test_dataset and perform the following steps:

- Concatenate the speaker ID and prediction to form the input text.
- Tokenize the input text using the tokenizer.
- Generate the output sequence using the trained model.
- Decode the output sequence to obtain the predicted sentence.
- Append the actual reference and the predicted sentence to the references and predictions lists, respectively.

In [ ]:
predictions = []
references = []

for example in test_dataset:
    # This creates an input text by combining the values of the 'speaker' and 'prediction' fields from the current example.
    # It assumes that the example is a dictionary-like object with keys 'speaker' and 'prediction'.
    input_text = f"{example['speaker']}: {example['prediction']}"

    # This creates a context where no gradients are computed, which can improve efficiency during inference.
    with torch.no_grad():

        # This tokenizes the input_text using the tokenizer and converts it into input IDs as a PyTorch tensor.
        input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

        # The generate method generates the output sequence based on the provided input.
        outputs = model.generate(input_ids=input_ids, max_length=max_length)

    predicted_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)

    references.append(example['actual'])
    predictions.append(predicted_sentence)

In [ ]:
# Verify that the number of predictions and references are the same
if len(predictions) == len(references):
    print("Number of predictions and references are the same.")
else:
    print("Mismatch in the number of predictions and references.")

In [ ]:
# Print the number of predictions and references
print("Number of predictions:", len(predictions))
print("Number of references:", len(references))

# print the length of the dataset
print("Number of rows in dataset:", len(test_dataset))

In [ ]:
# verification
print("prediction:", predictions)
print("reference:", references)

### **WER calculation for test speaker**

The code snippet calculates the Word Error Rate (WER) for the test speaker. It iterates over the predictions and references, comparing each prediction to its corresponding reference.

In [ ]:
# calculate WER

from jiwer import wer

wer_value = wer(predictions, references)
wer_percentage = wer_value * 100

print(f"WER: {wer_percentage:.2f}%")